<h1>Загрузка данных</h1>

In [48]:
import pandas as pd
import numpy as np
import sklearn

_csv = pd.read_csv("./data/train.csv")

_csv

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


<h1>Подготовка данных</h1><br>
Некоторые колонки удаляем за ненадобностью, такие как:<br>
'Fare' - Стоимость билета нас не интересует, поскольку нам уже известен класс каюты<br>
'Embarked' - Порт, в котором сел данный человек<br>
'Ticket' - Номер билета нас также не интересует<br>
'PassengerId' - Уникальный номер пассажира также нас не интересует<br>
'Cabin' - Потенциально важный параметр, однако в данной ситуации я его не учитываю<br>

In [49]:
_csv = _csv.drop(labels=["Fare", "Embarked", "Ticket", "PassengerId", "Cabin"], axis=1)

<h2>Обработка нулевых значений</h2><br>
Требуется проверить колонки на наличие нулевых\пропущенных значений, чтобы правильно их обработать.

In [50]:
_csv.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

Поскольку теперь мы знаем, что в колонке 'Sex' нет пропущенных значений, мы можем обработать её значения.<br>
Алгоритмы машинного обучения плохо работают со строками, поэтому все значения стоит преобразовывать в числа.<br>
В данном случае, женский пол ('female') мы обозначим '0', а мужской пол ('male') обозначим '1'.

In [51]:
_csv["Sex"] = _csv["Sex"].map({"female": 0, "male": 1})
_csv

,Survived,Pclass,Name,Sex,Age,SibSp,Parch
0,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0
2,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0
4,0,3,"Allen, Mr. William Henry",1,35.0,0,0
...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",1,27.0,0,0
887,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2
889,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0


Значения возраста 'Age' имеют пропуски, а также слишком разрозненны. Чтобы упростить классификацию, стоит объединить людей по возрастным группам, но для этого нужно узнать ограничения, чтобы группы были грамотно разделены.

In [52]:
_csv["Age"].min()

0.42

In [53]:
_csv["Age"].max()

80.0

Итоговые возрастные группы:<br>
1. 0-19<br>
2. 20-39<br>
3. 40-59<br>
4. 60-80<br>

Обработка нулевых значений будет осуществляться по следующему алгоритму:<br>
Если в имени женщины содержится Mrs. или Mme. (французский эквивалент Mrs.), то это означает, что она замужем, поэтому мы выдадим ей медианный возраст замужних девушек<br>
Если в имени девушки есть Miss. или Mlle. (французский эквивалент Miss.), то это значит, что она не замужем, поэтому мы выдадим ей медианный возраст незамужних девушек<br>
Если в имени девушки есть Ms., то мы выдадим ей медианный возраст всех девушек<br>

Если в имени мужчины есть Mr., то мы выдадим ему медианный возраст этой группы.<br>
Если в имени мужчины есть Master., то мы выдадим ему медианный возраст этой категории. Я выделил их отдельно, поскольку это звание есть чаще всего у детей<br>
Если в имени мужчины есть Don., Rev. Dr., то мы выдадим ему медианный возраст этой группы.<br>

In [54]:
g = _csv[~(_csv["Name"].str.contains("Mr.|Miss.|Mrs.|Ms."))]
#pd.set_option('display.max_rows', _csv.shape[0]+1)
g[["Name", "Sex", "Age"]]

,Name,Sex,Age
7,"Palsson, Master. Gosta Leonard",1,2.00
16,"Rice, Master. Eugene",1,2.00
30,"Uruchurtu, Don. Manuel E",1,40.00
50,"Panula, Master. Juha Niilo",1,7.00
59,"Goodwin, Master. William Frederick",1,11.00
...,...,...,...
831,"Richards, Master. George Sibley",1,0.83
848,"Harper, Rev. John",1,28.00
850,"Andersson, Master. Sigvard Harald Elias",1,4.00
869,"Johnson, Master. Harold Theodor",1,4.00


In [55]:
mrs_med = _csv[_csv["Name"].str.contains("Mrs.|Mme.")]["Age"].mean()
miss_med = _csv[_csv["Name"].str.contains("Miss.|Mlle.")]["Age"].mean()
ms_med = _csv[_csv["Name"].str.contains("Ms.")]["Age"].mean()

mr_med = _csv[_csv["Name"].str.contains("Mr.")]["Age"].mean()
master_med = _csv[_csv["Name"].str.contains("Master.")]["Age"].mean()
don_med = _csv[_csv["Name"].str.contains("Don.|Dr.|Rev.")]["Age"].mean()

# rounding
mrs_med = round(mrs_med, 2)
miss_med = round(miss_med, 2)
ms_med = round(ms_med, 2)
mr_med = round(mr_med, 2)
master_med = round(master_med, 2)
don_med = round(don_med, 2)

print(f"Медианные значения:\n'Mrs.'|'Mme.' - {mrs_med}\n'Miss.'|'Mlle.' - {miss_med}\n'Ms.' - {ms_med}\n'Mr.' - {mr_med}\n'Master.' - {master_med}\n'Don.'|'Dr.'|'Rev.' - {don_med}")

Медианные значения:
'Mrs.'|'Mme.' - 35.54
'Miss.'|'Mlle.' - 21.8
'Ms.' - 28.0
'Mr.' - 33.12
'Master.' - 4.57
'Don.'|'Dr.'|'Rev.' - 41.41


Обработка нулевых значений возраста

In [56]:
# Функция заполнения возрастных пропусков
def prep_ages(csv_file):
    csv_file.loc[csv_file["Name"].str.contains("Mrs.|Mme."), "Age"] = csv_file[csv_file["Name"].str.contains("Mrs.|Mme.")]["Age"].fillna(mrs_med)
    csv_file.loc[csv_file["Name"].str.contains("Miss.|Mlle."), "Age"] = csv_file[csv_file["Name"].str.contains("Miss.|Mlle.")]["Age"].fillna(miss_med)
    csv_file.loc[csv_file["Name"].str.contains("Ms."), "Age"] = csv_file[csv_file["Name"].str.contains("Ms.")]["Age"].fillna(ms_med)

    csv_file.loc[csv_file["Name"].str.contains("Mr."), "Age"] = csv_file[csv_file["Name"].str.contains("Mr.")]["Age"].fillna(mr_med)
    csv_file.loc[csv_file["Name"].str.contains("Master."), "Age"] = csv_file[csv_file["Name"].str.contains("Master.")]["Age"].fillna(master_med)
    csv_file.loc[csv_file["Name"].str.contains("Don.|Dr.|Rev."), "Age"] = csv_file[csv_file["Name"].str.contains("Don.|Dr.|Rev.")]["Age"].fillna(don_med)
    
    return csv_file

_csv = prep_ages(_csv)
_csv.isna().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
dtype: int64

In [57]:
# Функция группировки возраста
def group_ages(age):
    if age <= 19:
        age = 1
    elif age >= 20 and age <=39:
        age = 2
    elif age >= 40 and age <= 59:
        age = 3
    elif age >= 60 and age <= 80:
        age = 4
    return age

_csv["Age"] = _csv["Age"].apply(group_ages)
_csv

,Survived,Pclass,Name,Sex,Age,SibSp,Parch
0,0,3,"Braund, Mr. Owen Harris",1,2,1,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0
2,1,3,"Heikkinen, Miss. Laina",0,2,0,0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,2,1,0
4,0,3,"Allen, Mr. William Henry",1,2,0,0
...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",1,2,0,0
887,1,1,"Graham, Miss. Margaret Edith",0,1,0,0
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,2,1,2
889,1,1,"Behr, Mr. Karl Howell",1,2,0,0


In [58]:
train_labels = _csv["Survived"]
_csv = _csv.drop(labels=["Name", "Survived"], axis=1)

In [59]:
# Функция для общей подготовки данных
def prep_input(data):
    ids = data["PassengerId"]
    data = data.drop(labels=["Fare", "Embarked", "Ticket", "PassengerId", "Cabin"], axis=1)
    data["Sex"] = data["Sex"].map({"female": 0, "male": 1})
    data = prep_ages(data)
    data["Age"] = data["Age"].apply(group_ages)
    data = data.drop(labels=["Name"], axis=1)
    return data, ids

<h1>Сравнение алгоритмов</h1>

Поскольку данные для этой работы были взяты с сайта Kaggle, у меня нет доступа к итоговым значениям тестовых данных для того, чтобы оценить точность модели самостоятельно, поэтому оценка точности будет произведена с помощью submission полученных результатов на сайт.

In [60]:
_test_csv = pd.read_csv("./data/test.csv")
_test_csv, _test_ids = prep_input(_test_csv)
_test_csv

,Pclass,Sex,Age,SibSp,Parch
0,3,1,2,0,0
1,3,0,3,1,0
2,2,1,4,0,0
3,3,1,2,0,0
4,3,0,2,1,1
...,...,...,...,...,...
413,3,1,2,0,0
414,1,0,2,0,0
415,3,1,2,0,0
416,3,1,2,0,0


In [61]:
# Функция для сохранения итоговых данных в файл
def output_to_file(filename, ids, pred):
    df = pd.DataFrame({"PassengerId": ids, "Survived": pred})
    df.to_csv(filename, index=False)


<h2>Support Vector Machines</h2><br>
<h3>Support Vector Classification (SVC)</h3>

In [62]:
from sklearn import svm
clf = svm.SVC()
clf.fit(_csv, train_labels)
pred = clf.predict(_test_csv)
output_to_file("output/svm_svc_output.csv", _test_ids.values, pred)

<img src="img/svm_svc_score.png" title="Оценка точности с сайта Kaggle">

<h3>Support Vector Regression (SVR)</h3>

In [63]:
clf = svm.SVR()
clf.fit(_csv, train_labels)
pred = clf.predict(_test_csv)
output_to_file("output/svm_svr_output.csv", _test_ids.values, np.rint(pred).astype(int))

<img src="img/svm_svr_score.png" title="Оценка точности с сайта Kaggle">

<h2>Stochastic Gradient Descent (SGD)</h2>

In [64]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
clf.fit(_csv, train_labels)
pred = clf.predict(_test_csv)
output_to_file("output/linear_sgdclassifier_output.csv", _test_ids.values, pred)

<img src="img/linear_sgdclassifier_score.png" title="Оценка точности с сайта Kaggle">

<h2>Nearest Centroid Classifier</h2>

In [65]:
from sklearn.neighbors import NearestCentroid
clf = NearestCentroid()
clf.fit(_csv, train_labels)
pred = clf.predict(_test_csv)
output_to_file("output/nearestcentroid_output.csv", _test_ids.values, pred)

<img src="img/nearestcentroid_score.png" title="Оценка точности с сайта Kaggle">

<h2>Descision Tree</h2>

In [66]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(_csv, train_labels)
pred = clf.predict(_test_csv)
output_to_file("output/decisiontree_output.csv", _test_ids.values, pred)

<img src="img/decisiontree_score.png" title="Оценка точности с сайта Kaggle">

<h2>Вывод</h2><br>
На основании проведённого сравнения, в качестве итогового классификатора был выбран классификатор SVR.

<h1>Сохранение модели</h1>

In [67]:
import pickle5 as pickle
with open("models/svr_model.pckl", 'wb') as f:
    clf = svm.SVR()
    clf.fit(_csv, train_labels)
    pickle.dump(clf, f)